In [1]:
x=1

In [2]:

import scanpy as sc
import multigrate as mtg
import numpy as np

import tracemalloc
import scvi
import time
import torch

scvi.settings.seed = 0

Global seed set to 0
Global seed set to 0


In [3]:
torch.set_float32_matmul_precision('medium')

## multiome 25

In [4]:
tracemalloc.start()

In [5]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_gene_25.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_atac_25.h5ad')

In [6]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [7]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

In [8]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [9]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [10]:
adata

AnnData object with n_obs × n_vars = 17312 × 22000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [11]:
decay_at_n_obs = 20000
epochs_cap = 200
n_obs = adata.n_obs

max_epochs = min(round((decay_at_n_obs / n_obs) * epochs_cap), epochs_cap)
max_epochs = max(max_epochs, 1)
max_epochs

200

In [12]:
start_train_time = time.time()
vae.train(max_epochs=max_epochs)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 1/200:   0%|          | 0/200 [00:00<?, ?it/s]

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


Epoch 200/200: 100%|██████████| 200/200 [09:16<00:00,  2.73s/it, loss=1.35e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [09:16<00:00,  2.78s/it, loss=1.35e+03, v_num=1]


In [13]:
current, peak = tracemalloc.get_traced_memory()
print(f"Max epochs for {adata.n_obs} cells: {max_epochs}")
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)} MB")
tracemalloc.reset_peak()

Max epochs for 17312 cells: 200
Training time: 557.2225604057312 seconds
Peak training memory usage was 21855.575786590576 MB


## multiome 50

In [14]:
tracemalloc.start()

In [15]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_gene_50.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_atac_50.h5ad')

In [16]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [17]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

In [18]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [19]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [20]:
adata

AnnData object with n_obs × n_vars = 34624 × 22000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [21]:
decay_at_n_obs = 20000
epochs_cap = 200
n_obs = adata.n_obs

max_epochs = min(round((decay_at_n_obs / n_obs) * epochs_cap), epochs_cap)
max_epochs = max(max_epochs, 1)
max_epochs

116

In [22]:
start_train_time = time.time()
vae.train(max_epochs=max_epochs)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 116/116: 100%|██████████| 116/116 [10:20<00:00,  5.64s/it, loss=1.36e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=116` reached.


Epoch 116/116: 100%|██████████| 116/116 [10:20<00:00,  5.35s/it, loss=1.36e+03, v_num=1]


In [23]:
current, peak = tracemalloc.get_traced_memory()
print(f"Max epochs for {adata.n_obs} cells: {max_epochs}")
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)}MB")
tracemalloc.reset_peak()

Max epochs for 34624 cells: 116
Training time: 621.168089389801 seconds
Peak training memory usage was 22066.44140625MB


## multiome 75

In [24]:
tracemalloc.start()

In [25]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_gene_75.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/multiome/adata_atac_75.h5ad')

In [26]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [27]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

In [28]:
tracemalloc.start()

In [29]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [30]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [31]:
adata

AnnData object with n_obs × n_vars = 51936 × 22000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [32]:
decay_at_n_obs = 20000
epochs_cap = 200
n_obs = adata.n_obs

max_epochs = min(round((decay_at_n_obs / n_obs) * epochs_cap), epochs_cap)
max_epochs = max(max_epochs, 1)
max_epochs

77

In [33]:
start_train_time = time.time()
vae.train(max_epochs=max_epochs)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 77/77: 100%|██████████| 77/77 [10:19<00:00,  8.15s/it, loss=1.35e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=77` reached.


Epoch 77/77: 100%|██████████| 77/77 [10:19<00:00,  8.05s/it, loss=1.35e+03, v_num=1]


In [34]:
current, peak = tracemalloc.get_traced_memory()
print(f"Max epochs for {adata.n_obs} cells: {max_epochs}")
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)}MB")
tracemalloc.reset_peak()

Max epochs for 51936 cells: 77
Training time: 621.4292786121368 seconds
Peak training memory usage was 22473.459975242615MB


## 250k cells

In [35]:
tracemalloc.start()

In [36]:
old_n = np.array(adata.obs_names)

rng = 0
rng = np.random.default_rng(rng)

n = 250000
replace = True
indices = rng.choice(old_n, size=n, replace=replace)

adata = adata[indices].copy()
adata

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 250000 × 22000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [37]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=2000,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [38]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [39]:
adata

AnnData object with n_obs × n_vars = 250000 × 22000
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [40]:
decay_at_n_obs = 20000
epochs_cap = 200
n_obs = adata.n_obs

max_epochs = min(round((decay_at_n_obs / n_obs) * epochs_cap), epochs_cap)
max_epochs = max(max_epochs, 1)
max_epochs

16

In [41]:
start_train_time = time.time()
vae.train(max_epochs=max_epochs)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 16/16: 100%|██████████| 16/16 [09:39<00:00, 37.25s/it, loss=1.35e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=16` reached.


Epoch 16/16: 100%|██████████| 16/16 [09:39<00:00, 36.25s/it, loss=1.35e+03, v_num=1]


In [42]:
current, peak = tracemalloc.get_traced_memory()
print(f"Max epochs for {adata.n_obs} cells: {max_epochs}")
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)}MB")
tracemalloc.reset_peak()

Max epochs for 250000 cells: 16
Training time: 580.9321002960205 seconds
Peak training memory usage was 13280.174551010132MB
